In [3]:
import numpy as np
import pandas as pd
import os, gc, re

In [4]:
emails_df = pd.read_csv(r'emails.csv')
emails_df.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [5]:
emails_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517401 entries, 0 to 517400
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   file     517401 non-null  object
 1   message  517401 non-null  object
dtypes: object(2)
memory usage: 7.9+ MB


In [6]:
print(emails_df.iloc[22,1])

Message-ID: <26575732.1075855687756.JavaMail.evans@thyme>
Date: Mon, 2 Oct 2000 02:19:00 -0700 (PDT)
From: phillip.allen@enron.com
To: bs_stone@yahoo.com
Subject: Re: Original Sept check/closing
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: "BS Stone" <bs_stone@yahoo.com> @ ENRON
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Dec2000\Notes Folders\'sent mail
X-Origin: Allen-P
X-FileName: pallen.nsf

Brenda,

 Please use the second check as my October payment.  I have my copy of the 
original deal.  Do you want me to fax this to you?

Phillip


In [7]:
# split info
def info_part(i):
    return i.split('\n\n', 1)[0]

# split content
def content_part(i):
    return i.split('\n\n', 1)[1]

emails_df['pre_info'] = emails_df.message.map(info_part)
emails_df['content'] = emails_df.message.map(content_part)
emails_df['test_true'] = True

words2split = ['Message-ID: ', 'Date: ', 'From: ', 'To: ', 'Subject: ', 'Cc: ', 'Mime-Version: ', 'Content-Type: ',
               'Content-Transfer-Encoding: ', 'Bcc: ', 'X-From: ', 'X-To: ', 'X-cc: ', 'X-bcc: ', 'X-Folder: ', 'X-Origin: ',
               'X-FileName: ']
features_naming = [i[:-2] for i in words2split]
split_condition = '|'.join(words2split)

In [8]:
# emails subject confuse the string-spliting function<
def duplicated_info(i):
    return i.replace(' Date: ', ' Date- ').replace(' Subject: ', ' Subject2: ').replace(' To: ',
                    ' To- ').replace(' (Subject: ', ' (Subject- ')
emails_df['pre_info'] = emails_df['pre_info'].map(duplicated_info)

# let's check how many categories are there in these emails
def num_part(i):
    return len(re.split(split_condition, i))
emails_df['num_info'] = emails_df['pre_info'].map(num_part)

# around 20k emails do not have the 'To: ' category, so I add one
def add_to(i):
    return i.replace('\nSubject: ', '\nTo: \nSubject: ')
temp_condition = (emails_df['num_info'] == 17) | (emails_df['num_info'] == 15)
emails_df.loc[temp_condition, 'pre_info'] = emails_df.loc[temp_condition, 'pre_info'].map(add_to)

# similar way to deal with the "Cc:" and "Bcc:" categories
temp_condition = (emails_df['num_info'] == 16) | (emails_df['num_info'] == 15)

def add_bcc(i):
    return i.replace('\nX-From: ', '\nBcc: \nX-From: ')

emails_df.loc[temp_condition, 'pre_info'] = emails_df.loc[temp_condition, 'pre_info'].map(add_bcc)

def add_cc(i):
    return i.replace('\nMime-Version: ', '\nCc: \nMime-Version: ')

emails_df.loc[temp_condition, 'pre_info'] = emails_df.loc[temp_condition, 'pre_info'].map(add_cc)

In [9]:
emails_df['num_info'] = emails_df['pre_info'].map(num_part)
emails_df['num_info'].value_counts()

18    517398
5          2
6          1
Name: num_info, dtype: int64

In [10]:
df_remove = emails_df.loc[emails_df['num_info'] != 18].copy()
emails_df = emails_df.loc[emails_df['num_info'] == 18].copy()

In [11]:
global feature_idx

# split the i th part out and remove \n for the feature
def info_split(i):
    return re.split(split_condition, i)[feature_idx+1][:-2]

# no need to remove \n for last category -- X-FileName
def info_split_last(i):
    return re.split(split_condition, i)[feature_idx+1]
    
for feature_idx in range(len(words2split)):
    if feature_idx != len(words2split) - 1:
        emails_df[features_naming[feature_idx]] = emails_df['pre_info'].map(info_split)
    else:
        emails_df[features_naming[feature_idx]] = emails_df['pre_info'].map(info_split_last) 

In [12]:
emails_df['Content-Transfer-Encoding'].value_counts()

7bi                            494994
quoted-printabl                 22399
base6                               4
text/plain; charset=us-asci         1
Name: Content-Transfer-Encoding, dtype: int64

In [13]:
df_remove2 = emails_df.loc[emails_df['Content-Transfer-Encoding'] == 'text/plain; charset=us-asci']
emails_df = emails_df.loc[emails_df['Content-Transfer-Encoding'] != 'text/plain; charset=us-asci']

In [14]:
emails_df.loc[emails_df["content"].str.contains("-------------"), "content"]

9         ---------------------- Forwarded by Phillip K ...
12        ---------------------- Forwarded by Phillip K ...
13        ---------------------- Forwarded by Phillip K ...
16        ---------------------- Forwarded by Phillip K ...
18        ---------------------- Forwarded by Phillip K ...
                                ...                        
517175    \n\n -----Original Message-----\nFrom: \tKeoha...
517197    We can have you sit down with John Disturnal, ...
517205    \n\n -----Original Message-----\nFrom: \tkfrog...
517313    \n\n -----Original Message-----\nFrom: \t"Trav...
517321    \n\n -----Original Message-----\nFrom: \t"J&J ...
Name: content, Length: 97360, dtype: object

In [15]:
# split other forms of contents out
def split_other_content(i):
    return i.split('-------------', 1)[0]

emails_df["has_other_content"] = emails_df["content"].str.contains("-------------")
emails_df["if_forwarded"] = emails_df["content"].str.contains("------------- Forwarded")
emails_df['content'] = emails_df.content.map(split_other_content)

In [ ]:
emails_df = emails_df.drop(['pre_info','test_true', 'num_info'], axis = 1).set_index("file")
emails_df.to_csv("emails_cleaned.csv")